In [11]:
using JuMP, Ipopt, SparseArrays, LinearAlgebra, NamedArrays, DelimitedFiles, DataFrames, CSV, Statistics, PyPlot

# Include necessary functions
include("Functions/Ybus.jl");         # Function to calculate Ybus matrix (G,B)
include("Functions/Conventional.jl");   # Function to define constraints (polar)
include("Functions/Power_flow.jl");    # Function to calculate power flow
include("Functions/Cost.jl");          # Function to define cost (polar)
include("Functions/Misc.jl");                # Function to define start point,map input,print output
include("Functions/Parse_gld_1day.jl");      # Function to parse GridLabD system data
include("Functions/Fixed_point.jl");     # Function to implement fixed-point method
include("Functions/BFS.jl");             # Function to implement BFS method
include("Functions/Dist3Flow.jl");       # Function to implement Dist3Flow method
include("Functions/Active_constr.jl");   # Function to find active constraints

# Load test case
global case_name = "case13_test" 
#global case_name = "GC-12.47-1" 
# global case_name = "R1-12.47-3"  
#global case_name = "turkey_solar"
global PV_en = 0;   # 0-exclude, 1-include ZIP loads/PV inverters
if PV_en == 1
    house_sch = CSV.read(string("data/",case_name,"/House_",case_name,"_profile.csv"),header=false,type=Float64,DataFrame)
    global house_sch = convert(Matrix,house_sch);
    PV_sch = CSV.read(string("data/",case_name,"/PV_",case_name,"_profile.csv"),header=false,type=Float64,DataFrame)
    global PV_sch = convert(Matrix,PV_sch);
end
# global VUF_init = readdlm(string("VUF_",case_name,"_init.csv"), ',')/100
# global QgY_init = readdlm(string("QgY0_",case_name,"_base.csv"), ',');
# global QgY_init = readdlm("QgY0_R1-12.47-1_freq60_delay60.csv", ',');

## Other inputs
global count_sch = 684;  # Time instant to run OPF
# global crit_node = ["632"]
# global crit_node = ["l_49"]
# global crit_node = ["R1-12-47-1_node_359"]
# global crit_node = ["R1-12-47-3_node_1"]
global crit_node = "all"
global PV_rat = 0.035;
global Q_pen = 0*1e-5; global ΔV_pen = 0*1e-2;
global tap_en = 0; # Enable taps as variables only for ivr formulation
global obj_op = 2
       
## Parse system data
parse_gld(); global M = 1e3

## Create Y_bus and connection matrices
Ybus(); conn_bgl();

   # Initialize values
   start_point(4); global Vm0 = s0[1:nb_nph]; global θ0 = s0[nb_nph+1:2*nb_nph];
   global Vd0 = Vm0.*cos.(θ0); global Vq0 = Vm0.*sin.(θ0); global QgY0 = zeros(ngy_inv);
   stat = "LOCALLY_SOLVED";
   @time (Vm0,θ0,QgY0,stat) = TPOPF_ivr(crit_node,Vm0,θ0,QgY0)

par_det = 8×1 Named Adjoint{Int64, Matrix{Int64}}
Par ╲ Val │ :Value
──────────┼───────
nb        │      7
ngy       │      1
nly       │      2
nld       │      1
nb_nph    │     19
ngy_nph   │      3
nly_nph   │      5
nld_nph   │      1
-------------------
  0.027280 seconds (97.63 k allocations: 9.706 MiB)
LOCALLY_SOLVED
TOTAL TIME:
  5.082697 seconds (12.55 M allocations: 763.967 MiB, 3.37% gc time, 97.96% compilation time)


([0.9852934520882715, 0.9766231210907826, 0.9893499658451641, 0.9999965485764112, 0.9999912930525842, 0.9999961811186671, 0.9890509776978703, 0.9791126132838175, 0.9925878182990601, 0.999999999999997, 0.9999999999999933, 0.9999999999999972, 0.9553342466628945, 0.9529929684045839, 0.9660404673721663, 0.9683812713603915, 0.9911642063266961, 0.9667608192554468, 0.9892064109594222], [0.0017499761265431769, -2.1186703181806945, 2.0863593917953445, 2.442649464301874e-6, -2.0943902428155248, 2.094399333618145, 0.0031621458874963805, -2.117632321928741, 2.0864788361083333, -4.124515463875822e-17, -2.0943951023931953, 2.0943951023931953, -0.013472363322605568, -2.1295782778471968, 2.075737190832655, -2.1212047437304786, 2.0866146875101057, -2.122582401906464, 2.0873841053303157], Float64[], "LOCALLY_SOLVED")

In [46]:
v_ref = 0.5*(VR_config[1,3]+VR_config[1,4]);            
CT_s = 0.2; CT_p = VR_config[1,5]*CT_s;
R_comp = VR_config[1,6]/CT_s; X_comp = VR_config[1,7]/CT_s;
Z_comp = R_comp + im*X_comp; V0 = Vm0[kk].*exp.(im*θ0[kk]); i=6
j = Int(branch[i,1]); jj = Int(sum(bus_count[1:j-1])+1):Int(sum(bus_count[1:j]))
k = Int(branch[i,2]); kk = Int(sum(bus_count[1:k-1])+1):Int(sum(bus_count[1:k]))
Icomp_d = Ibus_d0[kk]*baseI[k]/VR_config[1,]; Icomp_q = Ibus_q0[kk]*baseI[k]/VR_config[1,6];
Vcomp_d = Vm0[kk].*cos.(θ0[kk])*bus_Vnom[k]/VR_config[1,2]; Vcomp_q = Vm0[kk].*sin.(θ0[kk])*bus_Vnom[k]/VR_config[1,2];
Icomp = Icomp_d+im*Icomp_q; Vcomp = Vcomp_d+im*Vcomp_q;
Vreg = Vcomp-Z_comp*Icomp
tap = (v_ref - abs.(Vreg))/0.75


3-element Vector{ComplexF64}:
 114.54080842647335 - 4.07156713243506im
 -63.52780947334282 - 88.14953229377133im
 -52.57499034262368 + 98.49895405213914im

In [50]:
abs.(Vreg)

3-element Vector{Float64}:
 114.6131513130321
 108.6559829005839
 111.65201994989786